## Fauna-ing Over Animal Detection

### Preperation

1. You'll want to make a directory called 'project_name' or whatever
2. Collect your data, you'll want pictures for each class, with bounding boxes and class labels etc. in XML files. People like [OpenCV](https://opencv.org/) but use whatever you like/want/can find the most stack overflow comments on.

### Downloads

These are the things you'll need to download to start using this repo. I'll try and make this step-by-step to get you up and running.

Clone the [tensorflow_things directory](https://github.com/MasonCaiby/Boulder_AI_CPW_study/tree/master/tensorflow_things) into your `project_name` directory

Clone the [tensorflow models/research repo](https://github.com/tensorflow/models) into the cloned `project_name/tensorflowthings` (this) directory. 
  follow the install instructions [from tensorflow](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md)

Navigate to models/research in your terminal and copy/paste this code into term (you can also add it to your Bash Profile): 

    ```export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim```

## Make the TFRecords

These records contain all of the data you will need to train your model. This means you will not need to upload your images to [AWS].

### First we need to get the images arranged correctly

The xml_to_csv will recursively check to make sure it gets all sub directories of `.../tensorflow_things/image_annotations/...`, so the structure of `.../images/` and `.../annotations/` doesn't matter too much

Put your training images in a folder called `.../tensorflow_things/image_annotations/images`

Put your training annotations in a folder called `.../tensorflow_things/image_annotations/annotations`. The name of the object for each bounding box can be in the xml file OR the folder name.

### Make CSV's of the bounding boxes

Run the ```xml_to_csv.py``` script. This will create a ```test.csv``` and ```train.csv``` file - the files wil be mixed but you must make sure you have the correct number of images for each class (e.g. balanced classes). How you obtain this is up to you: you can up sample, down sample, or just get the right number of images for each object.

In [1]:
run xml_to_csv.py

### Generate the TFRecords

We now need to make the tensorflow record files. These store your images and bounding box information. THIS MEANS YOU WILL NOT NEED TO UPLOAD YOUR TRAINING IMAGES TO AWS OR WHEREVER YOU CHOOSE TO TRAIN YOUR MODEL.

In [3]:
run generate_tfrecord.py  --csv_input=image_annotations/train.csv --output_path=train.record

/Users/maxcaudle/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/Users/maxcaudle/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 395876289 bytes but only got 0. Skipping tag 34464
  " Skipping tag %s" % (size, len(data), tag))
/Users/maxcaudle/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 12976128 bytes but only got 0. Skipping tag 2
  " Skipping tag %s" % (size, len(data), tag))
/Users/maxcaudle/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2686976 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
/Users/maxcaudle/anaconda3/lib/python3.6/site-package

Successfully created the TFRecords: /Users/maxcaudle/projects/capstone_backup/tensorflow_things/train2.record


In [6]:
# This will allow you to use generate_tfrecord.py again within this notebook. 
# Jupyter Notebook's store tf flags in memory and you can't change them (from what I can tell)
# once you've used the script once. We have made any variables so the reset is safe.

%reset -sf

In [5]:
run generate_tfrecord.py --csv_input=image_annotations/test.csv --output_path=test.record

/Users/maxcaudle/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/Users/maxcaudle/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 395876289 bytes but only got 0. Skipping tag 34464
  " Skipping tag %s" % (size, len(data), tag))
/Users/maxcaudle/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 12976128 bytes but only got 0. Skipping tag 2
  " Skipping tag %s" % (size, len(data), tag))
/Users/maxcaudle/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2686976 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
/Users/maxcaudle/anaconda3/lib/python3.6/site-package

Successfully created the TFRecords: /Users/maxcaudle/projects/capstone_backup/tensorflow_things/test2.record


I get some bad tag warnings, and I have ignored them. People online seem to do the same thing. My model still trained. The error is:

```
" Skipping tag %s" % (size, len(data), tag))
/Users/maxcaudle/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 35127296 bytes but only got 1080. Skipping tag 4
```

You should now have a `test.record` and `train.record` file in your `.../tensorflow_things/` directory.

## Train the model

### Get a pretrained model and config file

Sweet now you have your tfrecord files. Now we have to copy some things into the tensorflow_things directory. You'll adopt this for whichever model you wish to use (I used ```faster_rcnn_inception_resnet_v2_atrous_coco```).

    a. the config file for the model you want to use from [here](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs). 
  
    b. the tar'ed directory of the last checkpoint from [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)

### Edit the config file to what you need.

Change the config file to have the correct number of classes (for the faster rcnn it is num_classes) here:
```
model {
  faster_rcnn {
    num_classes: 3
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
```

Change the filepaths to point to the correct location, they are in all caps. For the [faster_rcnn](https://github.com/tensorflow/models/blob/master/research/object_detection/samples/configs/faster_rcnn_inception_resnet_v2_atrous_coco.config) I used they are:

#### a. line 108: 
    
```
fine_tune_checkpoint: "PATH_TO_BE_CONFIGURED/model.ckpt"
```  
to

```
fine_tune_checkpoint: "faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/model.ckpt"
```
        
#### b. line 123: 
```
input_path: "PATH_TO_BE_CONFIGURED/mscoco_train.record"
``` 
to 
```
input_path: "train.record"
```
       
#### c. line 125: 
```
label_map_path: "PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt"
``` 
to
```
label_map_path: "object-detection.pbtxt"
```
       
#### d. line 137: 
```
input_path: "PATH_TO_BE_CONFIGURED/mscoco_val.record"
``` 
to
```
input_path: "test.record"
```
       
#### e. line 139: 
```
label_map_path: "PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt"
``` 
to
```
label_map_path: "object-detection.pbtxt"
```

### Train the Model
You can run the model from your CL with:
`python models/research/object_detection/train.py --logtostderr --train_dir=training_faster_smaller_lr_1_3/ --pipeline_config_path=faster_rcnn_inception_resnet_v2_atrous_coco.config`

Or just run the cell below. (Note, this might choke your computer, depending on you know, your computer).

If you get a `ModuleNotFoundError: No module named 'object_detection'` error, you need to re-export the object detection directory. In your CL you need to go to `.../tensorflow_things/models/research/` directory and c/p : ```
export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
```

Note: If you are running this in a JN and are getting the module error, you'll have to export it then run JN from tensorflow_things in the same CL tab.

In [7]:
run models/research/object_detection/train.py --logtostderr --train_dir=training_directory/ --pipeline_config_path=faster_rcnn_inception_resnet_v2_atrous_coco.config

ModuleNotFoundError: No module named 'object_detection'

## Trained model

Now you need to let the model train. You can check the tensorboard here: `tensorboard --logdir='training_directory'`
When to stop letting the model train is up to you. You are, after all, the data scientist. I would urge you to wait until the error is <1, or maybe <<1. It's up to you though. Once you're satisfied with your model's progress, move to the next step.

Alternatively, you can move to the next step while your model is still training, check it on a few images, then either decide to stop training it or not.

### Creating a frozen model
This is pretty straight forward, thanks to TF's code base just c/p this command, after making sure the flags are set correctly (check: pipeline_config_path, trained_checkpoint_prefix, output_directory)

```
python3 object_detection/export_inference_graph.py \
--input_type image_tensor \
--pipeline_config_path ${PIPELINE_CONFIG_PATH} \
--trained_checkpoint_prefix ${TRAIN_PATH} \
--output_directory output_inference_graph.pb
```
An example of my code is:

```
python3 "models/research/object_detection/export_inference_graph.py" \
--input_type image_tensor \
--pipeline_config_path "faster_rcnn_inception_resnet_v2_atrous_coco1.config" \
--trained_checkpoint_prefix "training_faster1/model.ckpt-13972" \
--output_directory "full_inference_graph_maybe/"
```

I am choosing not to add this into a runable cell, because your variables will, well, vary. I did; however, include a blank Python cell below.


Congrats, you now have a model.

You can now use either the ```object_detection_tutorial.ipynb``` or ```test_pipe_aws.py``` file to test your images.

I'd suggest looking at the test_pipe_aws.py if you wish to use it. You can also contact me at: ```maxwell.caudle@gmail.com``` if you have any questions on anything.